In [1]:
import sys
sys.path.append('../')
import ta
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#machine learning/statistical dependencies
import statsmodels.api as sm
from kneed import KneeLocator
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, silhouette_score
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation
from sklearn.manifold import TSNE
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.arima.model import ARIMA 
# from pykalman import KalmanFilter
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Dropout
import scipy.cluster.hierarchy as shc
import plotly.express as px
import gc
from collections import defaultdict
import seaborn as sns
import lightgbm as lgb

2024-04-04 05:19:08.288021: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-04 05:19:08.984143: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from pair_trading_foundations.utils import GetSP500Data
import warnings
# The yfinance module has some deprecation warning
warnings.filterwarnings('ignore')

In [3]:
import requests
from datetime import datetime
earliest_date = '2008-12-01'
todays_date = datetime.today().strftime('%Y-%m-%d')
session = requests.Session()
session.verify = False
print(f'Extracting date range: [ {earliest_date} ]-[ {todays_date} ]')
all_data = GetSP500Data(earliest_date,todays_date).get_all_sp_tickers().get_consolidated_data();

Extracting date range: [ 2008-12-01 ]-[ 2024-04-04 ]


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [4]:
final_data = all_data.copy()
final_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,GICS Sector,GICS Sub-Industry
0,2008-12-01,54.306019,55.392979,51.923077,52.132107,29.470892,11619977.0,MMM,Industrials,Industrial Conglomerates
1,2008-12-02,50.752506,51.956524,49.180603,50.886288,28.766621,14273542.0,MMM,Industrials,Industrial Conglomerates
2,2008-12-03,49.882942,51.588631,49.464882,50.760872,28.695732,10731469.0,MMM,Industrials,Industrial Conglomerates
3,2008-12-04,50.275921,50.543476,48.578594,49.448162,27.953630,11111677.0,MMM,Industrials,Industrial Conglomerates
4,2008-12-05,48.670570,50.535118,48.361202,50.041805,28.289227,12285551.0,MMM,Industrials,Industrial Conglomerates


In [5]:
# Examine Null Values in final_data. Examine Date, Close and daily_returns.
final_data.isna().sum()

Date                 0
Open                 0
High                 0
Low                  0
Close                0
Adj Close            0
Volume               0
Ticker               0
GICS Sector          0
GICS Sub-Industry    0
dtype: int64

In [6]:
df = final_data.pivot(index='Date', columns='Ticker', values='Close') 
df.head()

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2008-12-01,12.346209,5.12,3.176071,NaN,NaN,24.061970,7.022222,29.059999,21.000000,16.100000,...,57.059605,33.950001,33.869999,17.690001,74.309998,NaN,17.951115,34.378639,18.980000,NaN
2008-12-02,12.618026,6.00,3.302500,NaN,NaN,24.935205,7.370000,30.020000,22.000000,16.350000,...,61.139072,34.480000,37.580002,17.889999,77.610001,NaN,18.138029,35.359222,19.590000,NaN
2008-12-03,13.018598,6.17,3.425000,NaN,NaN,25.501369,7.530000,30.870001,22.540001,17.299999,...,63.814571,36.189999,40.020000,18.190001,78.930000,NaN,19.439253,35.699028,19.500000,NaN
2008-12-04,12.525036,6.65,3.264643,NaN,NaN,24.963993,7.222222,29.129999,20.440001,16.190001,...,62.913906,35.419998,37.900002,17.570000,76.269997,NaN,19.920919,36.029125,18.110001,NaN
2008-12-05,12.346209,7.47,3.357143,NaN,NaN,25.309450,7.572222,28.270000,21.660000,16.660000,...,65.430466,35.389999,40.610001,18.180000,76.599998,NaN,20.848310,36.446602,18.510000,NaN


In [7]:
exp_data = df.copy()

# Null Values are Examined for Each Individual Stock on a Basis of Descriptive Statistics.

print('Null Values =',exp_data.isnull().values.any())

Null Values = True


In [8]:
 print('Data Shape Before Cleaning =', exp_data.shape)

missing_percentage = exp_data.isnull().mean().sort_values(ascending=False)
dropped_list = sorted(list(missing_percentage[missing_percentage > 0].index))
print(f"The following is a list of all stocks with ticker-level NA values: \n{dropped_list}")
exp_data.drop(labels=dropped_list, axis=1, inplace=True)

print('Data Shape After Cleaning =', exp_data.shape)

Data Shape Before Cleaning = (3860, 501)
The following is a list of all stocks with ticker-level NA values: 
['ABBV', 'ABNB', 'ALLE', 'AMCR', 'ANET', 'APTV', 'AVGO', 'CARR', 'CBOE', 'CDW', 'CEG', 'CFG', 'CHTR', 'CPAY', 'CTLT', 'CTVA', 'CZR', 'DAY', 'DG', 'DOW', 'ENPH', 'EPAM', 'ETSY', 'FANG', 'FOX', 'FOXA', 'FTNT', 'FTV', 'GEHC', 'GEV', 'GM', 'GNRC', 'GOOG', 'HCA', 'HII', 'HLT', 'HPE', 'HWM', 'INVH', 'IQV', 'IR', 'KEYS', 'KHC', 'KMI', 'KVUE', 'LW', 'LYB', 'META', 'MPC', 'MRNA', 'NCLH', 'NOW', 'NWS', 'NWSA', 'NXPI', 'OTIS', 'PANW', 'PAYC', 'PSX', 'PYPL', 'QRVO', 'SOLV', 'SYF', 'TRGP', 'TSLA', 'UBER', 'VICI', 'VLTO', 'VRSK', 'WRK', 'XYL', 'ZTS']
Data Shape After Cleaning = (3860, 429)


In [9]:
exp_data = exp_data.fillna(exp_data.median()) ### This step to ensure we did not miss any odd days, and if we did, it is standard industry practice to take the median

exp_data['year']=exp_data.index.year

exp_data

Ticker,A,AAL,AAPL,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP,...,WST,WTW,WY,WYNN,XEL,XOM,YUM,ZBH,ZBRA,year
Date,,,,,,,,,,,,,,,,,,,,,
2008-12-01,12.346209,5.12,3.176071,24.061970,7.022222,29.059999,21.000000,16.100000,25.549999,33.107990,...,16.520000,57.059605,33.950001,33.869999,17.690001,74.309998,17.951115,34.378639,18.980000,2008
2008-12-02,12.618026,6.00,3.302500,24.935205,7.370000,30.020000,22.000000,16.350000,26.120001,33.713783,...,17.105000,61.139072,34.480000,37.580002,17.889999,77.610001,18.138029,35.359222,19.590000,2008
2008-12-03,13.018598,6.17,3.425000,25.501369,7.530000,30.870001,22.540001,17.299999,25.100000,33.854259,...,17.450001,63.814571,36.189999,40.020000,18.190001,78.930000,19.439253,35.699028,19.500000,2008
2008-12-04,12.525036,6.65,3.264643,24.963993,7.222222,29.129999,20.440001,16.190001,24.610001,32.063213,...,17.355000,62.913906,35.419998,37.900002,17.570000,76.269997,19.920919,36.029125,18.110001,2008
2008-12-05,12.346209,7.47,3.357143,25.309450,7.572222,28.270000,21.660000,16.660000,25.830000,33.002632,...,18.270000,65.430466,35.389999,40.610001,18.180000,76.599998,20.848310,36.446602,18.510000,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-27,147.369995,15.30,173.309998,113.480003,91.480003,340.940002,504.399994,193.330002,62.990002,248.330002,...,394.100006,275.920013,35.930000,101.790001,53.380001,114.970001,137.320007,132.610001,299.160004,2024
2024-03-28,145.509995,15.35,171.479996,113.660004,92.440002,346.609985,504.600006,197.789993,62.810001,249.740005,...,395.709991,275.000000,35.910000,102.230003,53.750000,116.239998,138.649994,131.979996,301.440002,2024
2024-04-01,145.559998,15.41,170.029999,112.089996,92.309998,339.170013,502.089996,196.619995,62.349998,245.309998,...,395.940002,273.440002,35.560001,106.559998,53.279999,116.989998,140.470001,129.750000,301.000000,2024


In [10]:
# prepare clustering data by splitting train and test to match trading strategy later

cluster_train_size = int(len(exp_data) * 0.7)

# Split the data into training, validation and test sets based on these lengths:
cluster_train = exp_data[0:cluster_train_size]
print(f"The length of the training set is: {len(cluster_train)}.")

The length of the training set is: 2702.


In [11]:
max(cluster_train.index)

Timestamp('2019-08-26 00:00:00')

In [12]:
cluster_train

Ticker,A,AAL,AAPL,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP,...,WST,WTW,WY,WYNN,XEL,XOM,YUM,ZBH,ZBRA,year
Date,,,,,,,,,,,,,,,,,,,,,
2008-12-01,12.346209,5.120000,3.176071,24.061970,7.022222,29.059999,21.000000,16.100000,25.549999,33.107990,...,16.520000,57.059605,33.950001,33.869999,17.690001,74.309998,17.951115,34.378639,18.980000,2008
2008-12-02,12.618026,6.000000,3.302500,24.935205,7.370000,30.020000,22.000000,16.350000,26.120001,33.713783,...,17.105000,61.139072,34.480000,37.580002,17.889999,77.610001,18.138029,35.359222,19.590000,2008
2008-12-03,13.018598,6.170000,3.425000,25.501369,7.530000,30.870001,22.540001,17.299999,25.100000,33.854259,...,17.450001,63.814571,36.189999,40.020000,18.190001,78.930000,19.439253,35.699028,19.500000,2008
2008-12-04,12.525036,6.650000,3.264643,24.963993,7.222222,29.129999,20.440001,16.190001,24.610001,32.063213,...,17.355000,62.913906,35.419998,37.900002,17.570000,76.269997,19.920919,36.029125,18.110001,2008
2008-12-05,12.346209,7.470000,3.357143,25.309450,7.572222,28.270000,21.660000,16.660000,25.830000,33.002632,...,18.270000,65.430466,35.389999,40.610001,18.180000,76.599998,20.848310,36.446602,18.510000,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-20,71.120003,25.830000,52.590000,84.540001,39.770000,194.199997,285.709991,110.290001,37.480000,167.880005,...,145.119995,193.369995,25.030001,114.010002,62.400002,69.029999,116.570000,133.932037,201.440002,2019
2019-08-21,72.199997,26.420000,53.160000,84.900002,39.830002,195.729996,291.529999,108.589996,37.689999,169.690002,...,145.929993,193.750000,25.500000,113.559998,63.000000,69.720001,117.230003,133.427185,205.789993,2019
2019-08-22,71.739998,26.530001,53.115002,84.370003,39.570000,196.339996,287.690002,108.629997,37.930000,169.529999,...,145.860001,194.300003,25.639999,110.129997,63.279999,69.570000,117.309998,133.116501,204.380005,2019


In [13]:
# calculate average returns by year by stock
returns_year = cluster_train.groupby('year').mean()

# calculate return percent change, then aggregated returns and volatility by stock
returns = returns_year.pct_change().mean()

returns = pd.DataFrame(returns)
returns.columns = ['returns']
returns['volatility'] = returns_year.pct_change().dropna().std()

In [14]:
returns

,returns,volatility
Ticker,,
A,0.190269,0.161206
AAL,0.258942,0.569632
AAPL,0.309602,0.305773
ABT,0.117946,0.138215
ACGL,0.156064,0.142301
...,...,...
XEL,0.111120,0.062869
XOM,0.002344,0.105345
YUM,0.159641,0.095818


In [15]:
ret_data = returns.copy()
ret_data.describe()

,returns,volatility
count,429.000000,429.000000
mean,0.163448,0.200371
std,0.095418,0.113439
min,-0.061145,0.051727
25%,0.100850,0.124261
50%,0.150033,0.168138
75%,0.216263,0.242310
max,0.665196,0.707390


In [16]:
# Standardize The Clustering Dataset Along a t-Distributed Curve
scale = StandardScaler().fit(ret_data)

# Transform the Original Clustering Dataset to These Standardized Values
scaled_data = pd.DataFrame(scale.fit_transform(ret_data),columns = ret_data.columns, index = ret_data.index) 
X = scaled_data
X.describe()

,returns,volatility
count,4.290000e+02,4.290000e+02
mean,-2.650043e-16,-6.418072e-17
std,1.001168e+00,1.001168e+00
min,-2.356517e+00,-1.311868e+00
25%,-6.568023e-01,-6.717102e-01
50%,-1.407554e-01,-2.844762e-01
75%,5.541579e-01,3.701388e-01
max,5.264541e+00,4.474742e+00


In [17]:
n_clusters = 15
random_state = 15

# Algorithm:
# 1. Randomly pick k centroids from the examples ploted above as initial cluster centers (𝜇𝑗,𝑗∈1,...,𝑘)
# 2. Assign each example to the nearest centroid,  𝜇𝑗
# 3. Move the centroids to the center of the examples that were assigned to it
# 4. Repeat (2-3) until the cluster assignments do not change or a maximum number of iterations is reached.

def fit_kmeans(X, n_clusters, random_state):

    kmeans = KMeans(n_clusters=n_clusters,
                    init='k-means++',
                    n_init=15,
                    max_iter=300,
                    tol=1e-04)
    model = kmeans.fit(X)
    return model

model = fit_kmeans(X, n_clusters, random_state)
print(model)

## get cluster prediction and add to DataFrame
kX = X.copy()
kX['cluster'] = model.predict(X)
kX

KMeans(n_clusters=15, n_init=15)


,returns,volatility,cluster
Ticker,,,
A,0.281411,-0.345647,1
AAL,1.001959,3.258944,2
AAPL,1.533502,0.930237,3
ABT,-0.477423,-0.548558,7
ACGL,-0.077476,-0.512503,1
...,...,...,...
XEL,-0.549048,-1.213536,9
XOM,-1.690363,-0.838660,4
YUM,-0.039947,-0.922737,1


In [22]:
# c = kn.knee   # Number of clusters
c = 10   # Number of clusters - based on elbow method slope
#Fit the model
k_means = KMeans(n_clusters=c)
k_means.fit(X)
prediction = k_means.predict(X)

clustered_series = pd.Series(index=X.index, data=k_means.labels_.flatten())
clustered_series_all = pd.Series(index=X.index, data=k_means.labels_.flatten())
clustered_series = clustered_series[clustered_series != -1]

counts = clustered_series.value_counts()

cluster_vis_list = list(counts[(counts<20) & (counts>1)].index)[::-1]

clust1 = pd.DataFrame(clustered_series,columns=['label'])
grouped_df = clust1.groupby('label')

In [23]:
final_data_train = final_data.copy()
final_data_train = final_data_train.loc[final_data_train.Date <= max(cluster_train.index)]

In [24]:
pivot_data = final_data_train.pivot(index='Ticker', columns='Date', values='Adj Close')
pivot_data.head()

Date,2008-12-01,2008-12-02,2008-12-03,2008-12-04,2008-12-05,2008-12-08,2008-12-09,2008-12-10,2008-12-11,2008-12-12,...,2019-08-13,2019-08-14,2019-08-15,2019-08-16,2019-08-19,2019-08-20,2019-08-21,2019-08-22,2019-08-23,2019-08-26
Ticker,,,,,,,,,,,,,,,,,,,,,
A,11.096152,11.340450,11.700462,11.256876,11.096152,11.083297,11.526887,11.694037,11.655460,11.179729,...,65.434258,63.451115,67.320656,68.762077,69.410233,68.800781,69.845558,69.400558,67.281990,67.369034
AAL,4.826837,5.656449,5.816716,6.269231,7.042279,7.438230,6.429496,6.882013,6.146674,6.297513,...,27.320154,25.919884,25.234646,25.612024,25.919884,25.651747,26.237675,26.346916,25.244577,25.016165
AAPL,2.688859,2.795895,2.899604,2.763845,2.842155,3.015105,3.025383,2.969448,2.872391,2.971262,...,50.679306,49.170830,48.925888,50.080284,51.013985,51.016411,51.569347,51.525696,49.144154,50.077854
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,53.090603,51.857449,51.432789,52.616936,54.356407,54.233913,55.156738,55.140392,53.874584,54.274742
ABT,16.965704,17.581402,17.980593,17.601702,17.845276,17.902794,17.581402,17.429178,17.263411,17.168686,...,80.510887,77.777466,77.777466,78.555801,79.278526,78.333435,78.666977,78.175911,75.915031,77.017662


In [25]:
# cluster evaluation and model selection
cluster_size_limit = 9999
counts = clustered_series.value_counts()
ticker_count = counts[(counts>1) & (counts<=cluster_size_limit)]
print ("Number of clusters: %d" % len(ticker_count))
print ("Number of Pairs: %d" % (ticker_count*(ticker_count-1)).sum())

Number of clusters: 10
Number of Pairs: 25298


In [26]:
# Provide a functional definition for the cointegrated augmented dickey fuller test
def find_cadf_pairs(data):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = {}
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            ols = sm.OLS(S1,S2).fit()
            pred = ols.predict(S2)
            adf = ts.adfuller(S1-pred)
            score = adf[0]
            pvalue = adf[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.03:
                pairs[(keys[i], keys[j])] = adf
    return score_matrix, pvalue_matrix, pairs

In [27]:
# Extract Cointegrated Pairs. This takes approximately 30-40 minutes. Again, this would take much longer if clustering analysis was not applied.
adf_dict = {}

for i, clust in enumerate(ticker_count.index):
    tickers = clustered_series[clustered_series == clust].index
    score_matrix, pvalue_matrix, adf_pairs = find_cadf_pairs(cluster_train[tickers])
    adf_dict[clust] = {}
    adf_dict[clust]['score_matrix'] = score_matrix
    adf_dict[clust]['pvalue_matrix'] = pvalue_matrix
    adf_dict[clust]['pairs'] = adf_pairs
    
adf_pairs = []   
for cluster in adf_dict.keys():
    adf_pairs.extend(adf_dict[cluster]['pairs'])
    
print ("Number of pairs:", len(adf_pairs))
print ("In those pairs, we found %d unique tickers." % len(np.unique(adf_pairs)))
print(adf_pairs)

Number of pairs: 1159
In those pairs, we found 345 unique tickers.
[('ABT', 'ADI'), ('ABT', 'AJG'), ('ABT', 'BRO'), ('ABT', 'COST'), ('ABT', 'DRI'), ('ABT', 'NEE'), ('ABT', 'PLD'), ('ABT', 'TECH'), ('ABT', 'YUM'), ('ACGL', 'AIZ'), ('ACGL', 'CMCSA'), ('ACGL', 'CMS'), ('ACGL', 'EFX'), ('ACN', 'ADI'), ('ACN', 'ADP'), ('ACN', 'AON'), ('ACN', 'ATO'), ('ACN', 'AWK'), ('ACN', 'BDX'), ('ACN', 'CMS'), ('ACN', 'COST'), ('ACN', 'CRL'), ('ACN', 'DHR'), ('ACN', 'DRI'), ('ACN', 'ICE'), ('ACN', 'MKC'), ('ACN', 'NDAQ'), ('ACN', 'NEE'), ('ACN', 'WAT'), ('ACN', 'WM'), ('ACN', 'WRB'), ('ADI', 'ADP'), ('ADI', 'AIZ'), ('ADI', 'AJG'), ('ADI', 'ALL'), ('ADI', 'AON'), ('ADI', 'ATO'), ('ADI', 'AWK'), ('ADI', 'BDX'), ('ADI', 'BIO'), ('ADI', 'BRO'), ('ADI', 'CME'), ('ADI', 'CMS'), ('ADI', 'COST'), ('ADI', 'CRL'), ('ADI', 'DHR'), ('ADI', 'DRI'), ('ADI', 'DTE'), ('ADI', 'GL'), ('ADI', 'HON'), ('ADI', 'ICE'), ('ADI', 'JPM'), ('ADI', 'LLY'), ('ADI', 'LNT'), ('ADI', 'MCD'), ('ADI', 'MKC'), ('ADI', 'MMC'), ('ADI', 'NE

In [28]:
top500_pairs = list(map(list, adf_pairs))
top500_pairs = [lst for lst in top500_pairs if lst and not any(x is None for x in lst)]
top500_pairs

[['ABT', 'ADI'],
 ['ABT', 'AJG'],
 ['ABT', 'BRO'],
 ['ABT', 'COST'],
 ['ABT', 'DRI'],
 ['ABT', 'NEE'],
 ['ABT', 'PLD'],
 ['ABT', 'TECH'],
 ['ABT', 'YUM'],
 ['ACGL', 'AIZ'],
 ['ACGL', 'CMCSA'],
 ['ACGL', 'CMS'],
 ['ACGL', 'EFX'],
 ['ACN', 'ADI'],
 ['ACN', 'ADP'],
 ['ACN', 'AON'],
 ['ACN', 'ATO'],
 ['ACN', 'AWK'],
 ['ACN', 'BDX'],
 ['ACN', 'CMS'],
 ['ACN', 'COST'],
 ['ACN', 'CRL'],
 ['ACN', 'DHR'],
 ['ACN', 'DRI'],
 ['ACN', 'ICE'],
 ['ACN', 'MKC'],
 ['ACN', 'NDAQ'],
 ['ACN', 'NEE'],
 ['ACN', 'WAT'],
 ['ACN', 'WM'],
 ['ACN', 'WRB'],
 ['ADI', 'ADP'],
 ['ADI', 'AIZ'],
 ['ADI', 'AJG'],
 ['ADI', 'ALL'],
 ['ADI', 'AON'],
 ['ADI', 'ATO'],
 ['ADI', 'AWK'],
 ['ADI', 'BDX'],
 ['ADI', 'BIO'],
 ['ADI', 'BRO'],
 ['ADI', 'CME'],
 ['ADI', 'CMS'],
 ['ADI', 'COST'],
 ['ADI', 'CRL'],
 ['ADI', 'DHR'],
 ['ADI', 'DRI'],
 ['ADI', 'DTE'],
 ['ADI', 'GL'],
 ['ADI', 'HON'],
 ['ADI', 'ICE'],
 ['ADI', 'JPM'],
 ['ADI', 'LLY'],
 ['ADI', 'LNT'],
 ['ADI', 'MCD'],
 ['ADI', 'MKC'],
 ['ADI', 'MMC'],
 ['ADI', 'NEE'],
 ['AD

In [29]:
# Modification of the ADF test from earlier to apply to the top pairs in each cluster. Pulling out the p-value only.
def find_cadf_pairs_validate(data):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = {}
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            ols = sm.OLS(S1,S2).fit()
            pred = ols.predict(S2)
            adf = ts.adfuller(S1-pred)
            score = adf[0]
            pvalue = adf[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.05:
                pairs[(keys[i], keys[j])] = adf
    pvalue_result = min(pvalue_matrix[0])
    return pvalue_result

In [50]:
# loop the best cluster list to filter exp_data df
# loop the filtered df through find_cadf_pairs_validate function to retrieve the pvalue_result
# append the pvalue_result to a new dataframe to rank the best pair based on p-value

def cluster_ranking(data):
    n = len(data)
    cluster_ranked = pd.DataFrame({'ticker1':[],'ticker2':[],'pvalue':[]})
    for i in range(n):
        pair_validate_df=exp_data.filter(items=data[i])
        p_value = find_cadf_pairs_validate(pair_validate_df)
        cluster_ranked.loc[len(cluster_ranked)] = {'ticker1':top500_pairs[i][0],'ticker2':top500_pairs[i][1],'pvalue': p_value}
    cluster_ranked = cluster_ranked.sort_values(by='pvalue')
    return cluster_ranked


In [51]:
# display the best pair from each cluster, sorted by the lowest p-value
top500_df = cluster_ranking(top500_pairs)

In [52]:
top500_df = top500_df[0:500]

In [53]:
top500_df = top500_df.reset_index()

In [54]:
top500_df.to_csv('top500pairs.csv')